In [16]:
import re
from itertools import zip_longest

import numpy as np
from aocd.models import Puzzle
import pickle
import pyperclip

In [2]:
year, day = 2024, 9

In [3]:
puzzle = Puzzle(year=year, day=day)

In [4]:
puzzle.examples[0].answer_a

'1928'

In [5]:
example = puzzle.examples[0].input_data
print(example)

2333133121414131402


In [119]:
def solution_a(data: str) -> tuple[str, str, int]:
    # The disk map uses a dense format to represent the layout
    # of files and free space on the disk.
    # The digits alternate between indicating the length of a file
    # and the length of free space.
    # Each file on disk also has an ID number based on the order of the files
    # as they appear before they are rearranged, starting with ID 0.
    file_blocks = list(map(int, data[::2]))
    free_space_blocks = list(map(int, data[1::2]))
    FRAGMENTED_PATTERN = re.compile(r"\d\.+\d")

    block_representation = []

    for i, (file_block, free_space_block) in enumerate(
        zip_longest(file_blocks, free_space_blocks)
    ):
        if file_block:
            block_representation.append((i, file_block))

        if free_space_block:
            block_representation.append((".", free_space_block))

    # block_representation looks like [(0, 4), (., 3), (1, 4), (., 1)] = 4341

    str_block_representation = "".join("".join(str(x[0]) * x[1]) for x in block_representation)

    # continue defragmenting until all gaps are closes
    ordered_block_representation = block_representation.copy()
    leftmost_free_space_pos = 0
    rightmost_file_block_pos = len(ordered_block_representation) - 1
    while leftmost_free_space_pos <= rightmost_file_block_pos:
        #print(ordered_block_representation)
        # find next free space
        while True:
            if ordered_block_representation[leftmost_free_space_pos][0] == ".":
                break
            leftmost_free_space_pos += 1

        # find next block to move
        while True:
            if ordered_block_representation[rightmost_file_block_pos][0] != ".":
                break
            rightmost_file_block_pos -= 1

        # replace one "." with one number
        left, right = ordered_block_representation[leftmost_free_space_pos], ordered_block_representation[rightmost_file_block_pos]
        space_diff = left[1] - right[1]
        if space_diff == 0:
            ordered_block_representation[leftmost_free_space_pos] = (right[0], right[1])
            del ordered_block_representation[rightmost_file_block_pos]
            rightmost_file_block_pos -= 1 # we deleted one element
        elif space_diff > 0:
            ordered_block_representation[leftmost_free_space_pos] = (right[0], right[1])
            ordered_block_representation.insert(leftmost_free_space_pos+1, (".", space_diff))
            rightmost_file_block_pos += 1 # we added one element
            del ordered_block_representation[rightmost_file_block_pos]
            rightmost_file_block_pos -= 1 # we deleted one element
        else:
            space_diff *= (-1)
            ordered_block_representation[leftmost_free_space_pos] = (right[0], left[1])
            ordered_block_representation[rightmost_file_block_pos] = (right[0], space_diff)
            
    str_ordered_block_representation = "".join("".join(str(x[0]) * x[1]) for x in ordered_block_representation)
    
    checksum = 0
    pos = 0
    for block in ordered_block_representation:
        if block[0] == ".":
            break
        for _ in range(block[1]):
            checksum += block[0] * pos
            pos += 1

    return str_block_representation, str_ordered_block_representation, checksum

In [120]:
solution_a("12345")

('0..111....22222', '022111222', 60)

In [121]:
solution_a(puzzle.examples[0].input_data)

('00...111...2...333.44.5555.6666.777.888899',
 '0099811188827773336446555566.',
 1928)

In [124]:
assert (
    solution_a(puzzle.examples[0].input_data)[1].strip(".")
    == "0099811188827773336446555566"
)

In [125]:
assert solution_a(puzzle.examples[0].input_data)[2] == 1928

In [126]:
answer_a = solution_a(puzzle.input_data)

In [128]:
answer_a[2]

6200294120911

In [129]:
puzzle.answer_a = answer_a[2]

That's the right answer!  You are one gold star closer to finding the Chief Historian. [Continue to Part Two]


## Part Two


In [162]:
small_example = """T.........
...T......
.T........
..........
..........
..........
..........
..........
..........
.........."""

In [166]:
def solution_b(data: str) -> tuple[str, str, int]:
    # This time, attempt to move whole files to the leftmost span of free space blocks
    # that could fit the file. Attempt to move each file exactly once
    # in order of decreasing file ID number starting with the file with the highest file ID number. 
    # If there is no span of free space to the left of a file that is large enough to fit the file,
    # the file does not move.
    file_blocks = list(map(int, data[::2]))
    free_space_blocks = list(map(int, data[1::2]))
    FRAGMENTED_PATTERN = re.compile(r"\d\.+\d")

    block_representation = []

    for i, (file_block, free_space_block) in enumerate(
        zip_longest(file_blocks, free_space_blocks)
    ):
        if file_block:
            block_representation.append((i, file_block))

        if free_space_block:
            block_representation.append((".", free_space_block))

    # block_representation looks like [(0, 4), (., 3), (1, 4), (., 1)] = 4341

    str_block_representation = "".join("".join(str(x[0]) * x[1]) for x in block_representation)

    # continue defragmenting until all gaps are closes
    ordered_block_representation = block_representation.copy()
    
    # go through all file blocks from end to beginning
    right_pos = len(ordered_block_representation) - 1
    while right_pos > 0:
        # find next file block from end
        # at the end first block with pos 0 is always != "."
        while True:
            right = ordered_block_representation[right_pos]
            if right[0] != ".":
                break
            right_pos -= 1
        
        # find next free space block from beginning that can hold file block
        left_pos = 0
        while left_pos < right_pos:
            left = ordered_block_representation[left_pos]
            if left[0] == "." and left[1] >= right[1]:
                break
            left_pos += 1

        # if we cannot find a free space block, move to the next file block
        if left_pos >= right_pos:
            right_pos -= 1
            continue               

        # move whole file block into free space
        space_diff = left[1] - right[1]
        if space_diff == 0:
            ordered_block_representation[left_pos] = (right[0], right[1])
        elif space_diff > 0:
            ordered_block_representation[left_pos] = (right[0], right[1])
            ordered_block_representation.insert(left_pos + 1, (".", space_diff))
            right_pos += 1 # we inserted one element

        # by moving the whole block, we free the space at the original place
        ordered_block_representation[right_pos] = (".", right[1])
        
    str_ordered_block_representation = "".join("".join(str(x[0]) * x[1]) for x in ordered_block_representation)
    
    checksum = 0
    pos = 0
    for block in ordered_block_representation:
        if block[0] == ".":
            pos += block[1]
            continue
        for _ in range(block[1]):
            checksum += block[0] * pos
            pos += 1

    return str_block_representation, str_ordered_block_representation, checksum

In [167]:
solution_b(puzzle.examples[0].input_data)

('00...111...2...333.44.5555.6666.777.888899',
 '00992111777.44.333....5555.6666.....8888..',
 2858)

In [165]:
solution_b(puzzle.examples[0].input_data)[1].strip(".") == "00992111777.44.333....5555.6666.....8888"

00...111...2...333.44.5555.6666.777.888899
0099.111...2...333.44.5555.6666.777.8888..
0099.111...2...333.44.5555.6666.777.8888..
0099.1117772...333.44.5555.6666.....8888..
0099.1117772...333.44.5555.6666.....8888..
0099.1117772...333.44.5555.6666.....8888..
0099.111777244.333....5555.6666.....8888..
0099.111777244.333....5555.6666.....8888..
0099.111777244.333....5555.6666.....8888..
00992111777.44.333....5555.6666.....8888..
00992111777.44.333....5555.6666.....8888..
00992111777.44.333....5555.6666.....8888..
00992111777.44.333....5555.6666.....8888..


True

In [168]:
assert solution_b(puzzle.examples[0].input_data)[2] == 2858

In [ ]:
answer_b = solution_b(puzzle.input_data)
answer_b

In [170]:
puzzle.answer_b = answer_b[2]

That's the right answer!  You are one gold star closer to finding the Chief Historian.You have completed Day 9! You can [Shareon
  Bluesky
Twitter
Mastodon] this victory or [Return to Your Advent Calendar].
